In [78]:
import json
from src import dataio
from src import etri
from collections import Counter

# Basic settings

In [2]:
language = 'ko'
version = 1.0

In [3]:
from koreanframenet import koreanframenet
kfn = koreanframenet.interface(version)

### Korean FrameNet ###
	# contact: hahmyg@kaist, hahmyg@gmail.com #



In [105]:
from konlpy.tag import Kkma
kkma = Kkma()
def targetize(word):
    result = []
    morps = kkma.pos(word)
    v = False
    for m,p in morps:
        if p == 'XSV':
            v = True
    if v:
        for i in range(len(morps)):
            m,p = morps[i]
            if p == 'VA' or p == 'VV':
                if m[0] == word[0] and len(m) > 1:
                    result.append(m)
            if i > 0 and p == 'XSV':
                r = morps[i-1][0]+m
                if r[0] == word[0]:
                    result.append(r)
#     else:
    pos_list = []
    for m,p in morps:
        if p.startswith('J'):
            pos_list.append(m)
    for m, p in morps:
        if p == 'NNG':
            if len(pos_list) == 0:
                if m == word:
                    result.append(m)
            else:
                if m[0] == word[0]:
                    result.append(m)
    return result

with open('./data/targetdic.json','r') as f:
    targetdic = json.load(f)
def get_lu_by_token(token):
    target_candis = targetize(token)
    lu_candis = []
    for target_candi in target_candis:
        for lu in targetdic:
            if target_candi in targetdic[lu]:
                lu_candis.append(lu)
    common = Counter(lu_candis).most_common()
    if len(common) > 0:
        result = common[0][0]
    else:
        result = False
    return result

In [106]:
# input: text or json
def baseline(data):
    result = []
    idxs, tokens = data[0], data[1]
    for idx in range(len(tokens)):
        token = tokens[idx]
        lu = get_lu_by_token(token)
        lus = ['_' for i in range(len(tokens))]
        if lu:
            lus[idx] = lu
            instance = []            
#             instance.append(idxs)
            instance.append(tokens)
            instance.append(lus)
            result.append(instance)
    return result
        
text = '애플은 스티브 잡스와 스티브 워즈니악과 론 웨인이 1976년에 설립한 컴퓨터 회사이다.'
text = '헤밍웨이는 1961년 아이다호 주에서 62세의 나이에 자살했다.'
text = '헤밍웨이는 풀린 파이퍼와 이혼한 뒤 마사 겔혼과 재혼하였다'
tl = text.split(' ')
for t in tl:
    target = targetize(t)
    print(target)
# input_data = dataio.preprocessor(text)
# baseline(input_data)

[]
[]
['파이']
['이혼하', '이혼']
['뒤']
['마사']
[]
['재혼하', '재혼']
